In [1]:
import onnxruntime as ort
import glob as gb
import pandas as pd
import numpy as np
from utils.plotting_JPDF import plot_JPDF
import math 




In [2]:
def onnx_predict(session, X):
    # X: (N,2) float32
    input_name = session.get_inputs()[0].name
    out_names = [o.name for o in session.get_outputs()]
    preds = session.run(out_names, {input_name: X.astype(np.float32)})
    return preds[0]


In [3]:
# Read the data from the FlameMaster files
fname = './../neural_network/data/chemtable_FVV_2D_Enthalpy/*.kg'
files = gb.glob(fname)
nfiles = len(files)


# Read column names from the second line of the first file
with open(files[0], 'r') as f:
    lines = f.readlines()
    column_names = lines[1].strip().split('\t')  # Read second line (index 1) and split by whitespace

# Create empty lists to store the data
data_flameMaster = []

# Load and concatenate the data into DataFrames
for f in files:

    df_flameMaster_temp = pd.DataFrame(np.loadtxt(f, skiprows=2, dtype=np.float64),columns=column_names)
    data_flameMaster.append(df_flameMaster_temp)

# Concatenate all data into final DataFrames
df_flameMaster_all = pd.concat(data_flameMaster, ignore_index=True)

#Computing diffusivity 
df_flameMaster_all['Diff [kg/ms]'] = df_flameMaster_all['lambda [W/mK]'] / df_flameMaster_all['cp [J/kgK]']

# Select the relevant columns for input and output
input_data = ['ProgVar', 'TotalEnthalpy [J/kg]']
referenceEnthalpy = 276240

# Shift the enthalpy values to be relative to the reference enthalpy
df_flameMaster_all['TotalEnthalpy [J/kg]'] = df_flameMaster_all['TotalEnthalpy [J/kg]'] - referenceEnthalpy

# Select the relevant columns for input and output
output_data = ['ProdRateProgVar [kg/m^3s]', 'temperature [K]', 'Y-CO', 'density', 'mu [kg/ms]', 'cp [J/kgK]', 'Diff [kg/ms]']

# Create DataFrames for input and output data
X_all = df_flameMaster_all[input_data].to_numpy()
Z_all = df_flameMaster_all[output_data].to_numpy()

# Load mask from trained model
mask = np.load("./../neural_network/train_test_mask.npy")

X_train = X_all[mask].astype(np.float32)
Z_train = Z_all[mask].astype(np.float32)
X_test  = X_all[~mask].astype(np.float32)
Z_test  = Z_all[~mask].astype(np.float32)

x_mean = X_train.mean(axis=0).astype(np.float64)
x_std  = X_train.std(axis=0).astype(np.float64) + 1e-12


### ONNX model session

In [4]:
onnx_path = "./../neural_network/saved_models/NN_model_for_uq_analysis.onnx"
sess = ort.InferenceSession(onnx_path, providers=["CPUExecutionProvider"])

### Hermite Basis

In [5]:
def hermite_polynomial(n, x):
    if n == 0:
        return np.ones_like(x)
    elif n == 1:
        return x
    return x * hermite_polynomial(n-1, x) - (n-1) * hermite_polynomial(n-2, x)

def orthonormal_hermite(n, x):
    return hermite_polynomial(n, x) / np.sqrt(math.factorial(n))

def total_order_basis_2d(p):
    # list of (i,j) with i+j <= p
    return [(i, j) for i in range(p+1) for j in range(p+1) if i + j <= p]

def design_matrix_2d_hermite(Xs, basis):
    """
    Xs: standardized inputs (N,2)
    returns A: (N, P) where P = len(basis)
    """
    z1 = Xs[:, 0]
    z2 = Xs[:, 1]
    N = Xs.shape[0]
    P = len(basis)
    A = np.zeros((N, P), dtype=np.float64)

    # precompute H_n(z1), H_n(z2) up to max degree
    max_deg = max(max(i, j) for i, j in basis)
    H1 = np.stack([orthonormal_hermite(n, z1) for n in range(max_deg+1)], axis=1)  # (N, max_deg+1)
    H2 = np.stack([orthonormal_hermite(n, z2) for n in range(max_deg+1)], axis=1)

    for k, (i, j) in enumerate(basis):
        A[:, k] = H1[:, i] * H2[:, j]
    return A



### PCE Training

In [6]:
import numpy as np
import math
import time


# Choose polynomial order (start small)
order = 12  # try 3 or 4 first
sigma_rel = 0.05    # 5% relative uncertainty
rng = np.random.RandomState(0)

k=20
P = (order + 1) * (order + 2) // 2
N_fit = k * P  

rng = np.random.default_rng(0)

# 1) Draw xi samples from standard normal
XI_fit = rng.standard_normal(size=(N_fit, 2))

# 2) Choose a nominal point in physical space (here: training mean)
x0 = x_mean.copy()

# 3) Define physical-space standard deviations
#    Using relative uncertainty: sigma_phys = sigma_rel * |x0|
#    Add eps to avoid sigma=0 if x0 is near zero.
eps = 1e-8
sigma_phys = sigma_rel * np.maximum(np.abs(x0), eps)

# 4) Map xi -> X (physical)
X_fit_phys = x0 + XI_fit * sigma_phys

# 5) Clip to training-domain bounds to stay within interpolation regime
c_min, c_max = X_train[:, 0].min(), X_train[:, 0].max()
h_min, h_max = X_train[:, 1].min(), X_train[:, 1].max()

X_fit_phys[:, 0] = np.clip(X_fit_phys[:, 0], c_min, c_max)
X_fit_phys[:, 1] = np.clip(X_fit_phys[:, 1], h_min, h_max)

# 6) Evaluate the ONNX model at these physical inputs
Z_fit_phys = onnx_predict(sess, X_fit_phys)
Z_fit_phys = np.asarray(Z_fit_phys, dtype=np.float64)  # shape (N_fit, n_outputs)


### Fit PCE coefficients


In [7]:
basis = total_order_basis_2d(order)
print("order:", order, "| P terms:", len(basis))

A_fit = design_matrix_2d_hermite(XI_fit, basis)
print("A_fit shape:", A_fit.shape, "| Z_fit_phys shape:", Z_fit_phys.shape)

# Least squares fit for multi-output
coeffs, *_ = np.linalg.lstsq(A_fit, Z_fit_phys, rcond=None)  # shape (P, n_outputs)


order: 12 | P terms: 91
A_fit shape: (1820, 91) | Z_fit_phys shape: (1820, 7)


In [9]:

# ==============================================================
# Mean and variance from orthonormal PCE coefficients
# For orthonormal basis under N(0,1):
#   mean = c0
#   var  = sum_{k>0} c_k^2
# ==============================================================
mean_pce = coeffs[0, :]                      # (n_outputs,)
var_pce  = np.sum(coeffs[1:, :]**2, axis=0)  # (n_outputs,)
std_pce  = np.sqrt(var_pce)

df_uq = pd.DataFrame({
    "Output": output_data,
    "Mean": mean_pce,
    "Variance": var_pce,
    "Std": std_pce
})

df_uq


# ==============================================================
# Extra safety check (optional):
# Compare coefficient-based variance vs sample-based variance of the PCE surrogate
# ==============================================================
# Y_hat = A_fit @ coeffs
# mean_mc_pce = Y_hat.mean(axis=0)
# var_mc_pce  = Y_hat.var(axis=0, ddof=1)

# df_check = pd.DataFrame({
#     "Output": output_data,
#     "Mean_coeff": mean_pce,
#     "Mean_mc": mean_mc_pce,
#     "Var_coeff": var_pce,
#     "Var_mc": var_mc_pce,
# })

# print("\nSanity check (coeff-based vs sample-based):")
# df_check

df_uq

,Output,Mean,Variance,Std
0,ProdRateProgVar [kg/m^3s],349.208756,3.577239e+04,189.135901
1,temperature [K],1464.723516,1.602384e+03,40.029794
2,Y-CO,0.010566,1.406820e-07,0.000375
3,density,0.931452,6.602448e-04,0.025695
4,mu [kg/ms],0.000055,2.629792e-11,0.000005
5,cp [J/kgK],1321.378143,2.684026e+01,5.180759
6,Diff [kg/ms],0.000070,3.514459e-11,0.000006
